# Westeros Tutorial Part 2b - Firm Capacity

In the other tutorials (``westeros_emissions_bounds`` and ``westeros_emissions_taxes``), we showed how to introduce emissions into a stylized energy system model, and what happens if you put a constraint on total CO2 emissions.

In this tutorial notebook, we will illustrate how to add other constraints, in particular the requirement to have sufficient dispatchable (firm) capacity.

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully

In [ ]:
import pandas as pd
import ixmp
import message_ix

from message_ix.util import make_df

%matplotlib inline

In [ ]:
mp = ixmp.Platform()

In [ ]:
base = message_ix.Scenario(mp, model='Westeros Electrified', scenario='emission_bound')

In [ ]:
model = 'Westeros Electrified'
scen = base.clone(model, 'firm_capacity','illustration of firm-capacity formulation',
                  keep_solution=False)
scen.check_out()

year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']
model_horizon = scen.set('year')
country = 'Westeros'

## Improving the Representation of Electricity Sector
### Peak load factor
The input demand for electricity (notice: not the demand for useful light but for electricity) in the model shows the average
electricity demand in the given time (which in our example is a year).
However, power systems need enough installed capacity not only to cover the average electricity demand but also to meet the peak
demand, i.e., maximum load throughout that time.
This feature can be specified in the model using parameter `peak_load_factor`.
For example, if annual average load is 5 GW, the peak load may be twice as high, at 10 GW; hence, the peak load factor is equal
to 2.

In [ ]:
# peak_load_factor(node,commodity,level,year,time)
peak_load_factor = pd.DataFrame({
        'node': country,
        'commodity': 'electricity',
        'level' : 'secondary',       
        'year': model_horizon,
        'time' : 'year',
        'value' : 2,
        'unit' : '-'})

scen.add_par('peak_load_factor', peak_load_factor)

## Reliability of Power System
In order to meet demand reliably, the power system needs to maintain dispatchable or so-called firm capacity at any time.
Some technologies like coal power plants can fully contribute to the required firm capacity, i.e.,
1 MW installed capacity of these technologies provide 1 MW firm capacity.
However, some other technologies such as variable renewables like wind are not fully reliable when the power system needs them.
As such, the capacity value of wind power plant is considered to be lower than 1.
In this example, we assume that 10% of electricity supply by wind is 80% reliable, while the remaining 90% installed capacity can only contribute by 5% to the required firm capacity. As such, we divide wind power into two parts with two different ratings (r1 and r2) and we define this through parameter `rating_bin`.


In [ ]:
base_reliability = pd.DataFrame({
        'node': country,
        'commodity': 'electricity',
        'level' : 'secondary', 
        'unit': '-',
        'time': 'year',
        'year_act': model_horizon})

In [ ]:
# adding wind ratings to the respective set 
scen.add_set('rating', ['r1', 'r2'])

# adding rating bins for wind power plant
rating_bin = make_df(base_reliability, technology= 'wind_ppl', value = 0.1, rating= 'r1')
scen.add_par('rating_bin', rating_bin)

rating_bin = make_df(base_reliability, technology= 'wind_ppl', value = 0.9, rating= 'r2')
scen.add_par('rating_bin', rating_bin)

# adding reliability factor for each rating of wind power plant
reliability_factor = make_df(base_reliability, technology= 'wind_ppl', value = 0.8, rating= 'r1')
scen.add_par('reliability_factor', reliability_factor)

reliability_factor = make_df(base_reliability, technology= 'wind_ppl', value = 0.05, rating= 'r2')
scen.add_par('reliability_factor', reliability_factor)

# considering coal power plant as firm capacity (adding a reliability factor of 1)
reliability_factor = make_df(base_reliability, technology= 'coal_ppl', value = 1, rating= 'firm')
scen.add_par('reliability_factor', reliability_factor)

## Commit and Solve

In [ ]:
scen.commit(comment='define parameters for renewable implementation')
scen.set_as_default()

In [ ]:
scen.solve()

In [ ]:
scen.var('OBJ')['lvl']

## Plotting

In [ ]:
from message_ix.reporting import Reporter
from message_ix.util.tutorial import prepare_plots

rep = Reporter.from_scenario(scen)
prepare_plots(rep)

In [ ]:
rep.set_filters(t=["coal_ppl", "wind_ppl"])
rep.get("plot activity")

In [ ]:
rep.get("plot capacity")

In [ ]:
rep.set_filters(t=None, c=["light"])
rep.get("plot prices")

## Close the connection to the database

In [ ]:
mp.close_db()